In [10]:
import pandas as pd
import numpy as np
from tqdm import tqdm

csv_path = r"C:\Users\Jouke\Documents\evedata-logger\output\update_2025-07-13.csv"
df = pd.read_csv(csv_path, parse_dates=['date'])

ma_windows = list(range(7, 70, 7))
strategies = ['MA', 'EMA', 'WMA', 'RSI', 'MACD', 'BB']
results = []

unique_items = df['type_name'].unique()
print(f"Scanning {len(unique_items)} unique items...")
pbar = tqdm(total=len(unique_items) * len(ma_windows) * len(strategies), desc="Sweep+OOS", ncols=90)

def get_signals(item_df, strat, window):
    # --- Calculate indicators as needed ---
    if strat == 'MA':
        item_df['MA'] = item_df['average'].rolling(window=window, min_periods=1).mean()
        signal_col = 'MA'
    elif strat == 'EMA':
        item_df['EMA'] = item_df['average'].ewm(span=window, adjust=False).mean()
        signal_col = 'EMA'
    elif strat == 'WMA':
        weights = np.arange(1, window+1)
        item_df['WMA'] = item_df['average'].rolling(window, min_periods=1).apply(
            lambda x: np.dot(x, weights[-len(x):]) / weights[-len(x):].sum(), raw=True)
        signal_col = 'WMA'
    elif strat == 'RSI':
        delta = item_df['average'].diff()
        up = delta.clip(lower=0)
        down = -delta.clip(upper=0)
        roll_up = up.rolling(window, min_periods=1).mean()
        roll_down = down.rolling(window, min_periods=1).mean()
        rs = roll_up / roll_down.replace(0, np.nan)
        item_df['RSI'] = 100 - (100 / (1 + rs))
        signal_col = 'RSI'
    elif strat == 'MACD':
        ema_fast = item_df['average'].ewm(span=12, adjust=False).mean()
        ema_slow = item_df['average'].ewm(span=26, adjust=False).mean()
        item_df['MACD'] = ema_fast - ema_slow
        item_df['MACD_signal'] = item_df['MACD'].ewm(span=9, adjust=False).mean()
        signal_col = 'MACD'
    elif strat == 'BB':
        ma = item_df['average'].rolling(window=window, min_periods=1).mean()
        std = item_df['average'].rolling(window, min_periods=1).std()
        item_df['BB_upper'] = ma + 2 * std
        item_df['BB_lower'] = ma - 2 * std
        signal_col = 'BB'
    return item_df, signal_col

def simulate_trades(item_df, strat, window):
    # --- Simulate buys/sells ---
    if strat in ['MA', 'EMA', 'WMA']:
        sig = item_df['average'] > item_df[strat]
        buy_idx = sig & (~sig.shift(1, fill_value=False))
        sell_idx = ~sig & (sig.shift(1, fill_value=False))
    elif strat == 'RSI':
        buy_idx = item_df['RSI'] < 30
        sell_idx = item_df['RSI'] > 70
    elif strat == 'MACD':
        buy_sig = item_df['MACD'] > item_df['MACD_signal']
        sell_sig = item_df['MACD'] < item_df['MACD_signal']
        buy_idx = buy_sig & (~buy_sig.shift(1, fill_value=False))
        sell_idx = sell_sig & (~sell_sig.shift(1, fill_value=False))
    elif strat == 'BB':
        buy_idx = item_df['average'] < item_df['BB_lower']
        sell_idx = item_df['average'] > item_df['BB_upper']
    else:
        return 0, 0, 0

    buy_prices = item_df.loc[buy_idx, 'average'].values
    sell_prices = item_df.loc[sell_idx, 'average'].values
    n_trades = min(len(buy_prices), len(sell_prices))
    if n_trades > 0:
        profit = (sell_prices[:n_trades] - buy_prices[:n_trades]).sum()
        roi = profit / buy_prices[:n_trades].sum() if buy_prices[:n_trades].sum() > 0 else np.nan
        return profit, roi, n_trades
    return 0, 0, 0

robust_results = []
for item in unique_items:
    item_df_full = df[df['type_name'] == item].sort_values('date').reset_index(drop=True)
    if len(item_df_full) < max(ma_windows)*2:  # Need enough history for OOS
        pbar.update(len(ma_windows)*len(strategies))
        continue

    split_idx = len(item_df_full) // 2
    train_df = item_df_full.iloc[:split_idx].copy()
    test_df = item_df_full.iloc[split_idx:].copy()

    for strat in strategies:
        for w in ma_windows:
            # --- TRAIN ---
            tdf, _ = get_signals(train_df.copy(), strat, w)
            t_profit, t_roi, t_trades = simulate_trades(tdf, strat, w)
            # --- TEST (OOS) ---
            testdf, _ = get_signals(test_df.copy(), strat, w)
            v_profit, v_roi, v_trades = simulate_trades(testdf, strat, w)

            # --- Only keep combos with >30 trades and ROI > 0 on both sets
            if (t_trades >= 30 and v_trades >= 30) and (t_roi > 0 and v_roi > 0):
                robust_results.append({
                    'item': item, 'strategy': strat, 'window': w,
                    'train_roi': t_roi, 'train_trades': t_trades,
                    'test_roi': v_roi, 'test_trades': v_trades,
                    'train_profit': t_profit, 'test_profit': v_profit
                })
            pbar.update(1)
pbar.close()

robust_df = pd.DataFrame(robust_results)
print(f"\nRobust out-of-sample strategies found: {len(robust_df)}")
print(robust_df.sort_values('test_roi', ascending=False).head(30))

# Save for further analysis
robust_df.to_csv(r"C:\Users\Jouke\Documents\evedata-logger\output\robust_OOS_strategies.csv", index=False)
print("\nOOS results saved for review.")


Scanning 14104 unique items...


Sweep+OOS: 100%|█████████████████████████████████| 761616/761616 [44:30<00:00, 285.21it/s]


Robust out-of-sample strategies found: 300
                                       item strategy  window  train_roi  \
247  Medium Transverse Bulkhead I Blueprint      RSI      28   1.187941   
246  Medium Transverse Bulkhead I Blueprint      RSI      21   1.284201   
130                             Force Cable      RSI      14   0.825934   
150    Large Trimark Armor Pump I Blueprint      RSI      14   0.571863   
81                    Heat Sink I Blueprint      RSI      14   1.663410   
193       Small Gravity Capacitor Upgrade I      RSI       7   0.087675   
149    Large Trimark Armor Pump I Blueprint      RSI       7   0.149592   
131          Medium Armor Maintenance Bot I      RSI       7   0.372364   
89                                 Aurora S      RSI       7   0.247247   
285                  Meson Exotic Plasma XL      RSI       7   0.099441   
58                   Standard Crash Booster      RSI       7   0.174670   
210        Small Hydraulic Bay Thrusters II      RSI    

In [15]:
import pandas as pd

# Load the CSV
csv_path = r"C:\Users\Jouke\Documents\evedata-logger\output\robust_OOS_strategies.csv"
df = pd.read_csv(csv_path)

# Peek at data
print(df.head())
print(df.info())

# Only strategies with at least 30 trades in OOS
robust = df[df['test_trades'] >= 30].copy()
top_20 = robust.sort_values('test_roi', ascending=False).head(20)
print(top_20[['item', 'strategy', 'window', 'train_roi', 'test_roi', 'train_trades', 'test_trades']])

print(f"Total robust strategies: {len(robust)}")
print(f"Unique items: {robust['item'].nunique()}")
print("Strategy breakdown:")
print(robust['strategy'].value_counts())

print(robust[['test_roi', 'test_trades']].describe())

top_robust_path = r"C:\Users\Jouke\Documents\evedata-logger\output\robust_OOS_strategies_top.csv"
top_20.to_csv(top_robust_path, index=False)
print(f"Top robust OOS strategies saved to {top_robust_path}")






                       item strategy  window  train_roi  train_trades  \
0  Small Shield Extender II      RSI      14   0.017020            46   
1  Small Shield Extender II      RSI      21   0.082801            37   
2        1MN Afterburner II      RSI       7   0.113962            44   
3     5MN Microwarpdrive II      RSI       7   0.050454            48   
4                  Miner II      RSI       7   0.010979            34   

   test_roi  test_trades  train_profit  test_profit  
0  0.245432           48      331050.0    4743150.0  
1  0.290251           48     1270950.0    5414450.0  
2  0.084035           42    10473500.0    8862500.0  
3  0.032829           35     9396000.0    5556000.0  
4  0.026022           32      273100.0     693350.0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   item          300 non-null    object 
 1  

In [20]:
import pandas as pd
import numpy as np
import os

# --- Paths ---
oos_path = r"C:\Users\Jouke\Documents\evedata-logger\output\robust_OOS_strategies_top.csv"
hist_path = r"C:\Users\Jouke\Documents\evedata-logger\output\update_2025-07-13.csv"
out_path = r"C:\Users\Jouke\Documents\evedata-logger\output\signals_robust_RSI_gt5pct.csv"

# --- Load data ---
results = pd.read_csv(oos_path)
history = pd.read_csv(hist_path, parse_dates=['date'])

# --- Filter for >5% test ROI ---
filtered = results[results['test_roi'] > 0.05].reset_index(drop=True)
print(f"Robust strategies with test ROI > 5%: {len(filtered)}")

signal_records = []

for idx, row in filtered.iterrows():
    item = row['item']
    window = int(row['window'])
    strat = row['strategy']
    # Only RSI is present in your OOS CSV
    item_hist = history[history['type_name'] == item].sort_values('date').copy()
    if len(item_hist) < window+2:
        continue
    # --- Compute RSI ---
    delta = item_hist['average'].diff()
    up = delta.clip(lower=0)
    down = -delta.clip(upper=0)
    roll_up = up.rolling(window, min_periods=1).mean()
    roll_down = down.rolling(window, min_periods=1).mean()
    rs = roll_up / roll_down.replace(0, 1e-10)
    item_hist['RSI'] = 100 - (100 / (1 + rs))

    # --- Extract buy/sell signals (classic 30/70) ---
    item_hist['signal'] = 0
    item_hist.loc[item_hist['RSI'] < 30, 'signal'] = 1   # Buy
    item_hist.loc[item_hist['RSI'] > 70, 'signal'] = -1  # Sell

    # Only log rows where signal != 0
    for _, sig_row in item_hist[item_hist['signal'] != 0].iterrows():
        signal_records.append({
            'item': item,
            'strategy': strat,
            'window': window,
            'date': sig_row['date'],
            'action': 'BUY' if sig_row['signal'] == 1 else 'SELL',
            'price': sig_row['average'],
            'RSI': sig_row['RSI']
        })

# --- Save as summary CSV ---
signals_df = pd.DataFrame(signal_records)
signals_df.sort_values(['item', 'date'], inplace=True)
signals_df.to_csv(out_path, index=False)
print(f"Signals for all robust (>5% test ROI) strategies saved to:\n{out_path}")

print(signals_df.head())
print(f"\nTotal signals extracted: {len(signals_df)} (across {signals_df['item'].nunique()} unique items)")


Robust strategies with test ROI > 5%: 20
Signals for all robust (>5% test ROI) strategies saved to:
C:\Users\Jouke\Documents\evedata-logger\output\signals_robust_RSI_gt5pct.csv
          item strategy  window       date action     price         RSI
1376  Aurora S      RSI       7 2024-07-14   SELL  139500.0  100.000000
1377  Aurora S      RSI       7 2024-07-15   SELL  138900.0   98.619102
1378  Aurora S      RSI       7 2024-07-22    BUY   86180.0   27.856183
1379  Aurora S      RSI       7 2024-07-26    BUY   70090.0   18.626820
1380  Aurora S      RSI       7 2024-08-08   SELL   68380.0   94.029851

Total signals extracted: 3565 (across 18 unique items)


In [26]:
import pandas as pd

# --- Paths ---
signals_path = r"C:\Users\Jouke\Documents\evedata-logger\output\signals_robust_RSI_gt5pct.csv"
results_path = r"C:\Users\Jouke\Documents\evedata-logger\output\robust_OOS_strategies_top.csv"

# --- Load ---
signals = pd.read_csv(signals_path, parse_dates=['date'])
results = pd.read_csv(results_path)

# --- Only 5%+ test ROI ---
robust = results[results['test_roi'] > 0.05]

summary_rows = []
for i, row in robust.iterrows():
    item = row['item']
    window = int(row['window'])
    strat = row['strategy']
    roi = row['test_roi']
    profit = row['test_profit']
    n_trades = row['test_trades']
    # Get signals for this item+window
    mask = (signals['item'] == item) & (signals['window'] == window)
    s = signals[mask]
    n_buys = (s['action'] == 'BUY').sum()
    n_sells = (s['action'] == 'SELL').sum()
    first_trade = s['date'].min() if not s.empty else "N/A"
    last_trade = s['date'].max() if not s.empty else "N/A"
    summary_rows.append({
        "Item": item,
        "Strategy": strat,
        "Win": window,
        "ROI": f"{roi:.2%}",
        "Profit": f"{profit:,.0f}",
        "Trades": int(n_trades),
        "BUYs": int(n_buys),
        "SELLs": int(n_sells),
        "First Signal": first_trade,
        "Last Signal": last_trade,
    })

summary_df = pd.DataFrame(summary_rows)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 200)

print("\n=== Robust RSI Strategies (>5% test ROI) ===\n")
print(summary_df.to_string(index=False))

summary_df.to_csv(r"C:\Users\Jouke\Documents\evedata-logger\output\robust_rsi_summary_readable.csv", index=False)




=== Robust RSI Strategies (>5% test ROI) ===

                                  Item Strategy  Win     ROI      Profit  Trades  BUYs  SELLs First Signal Last Signal
Medium Transverse Bulkhead I Blueprint      RSI   28 222.51% 110,375,581      33    57     73   2024-06-02  2025-06-20
Medium Transverse Bulkhead I Blueprint      RSI   21 205.42% 108,673,366      34    77     75   2024-06-02  2025-06-26
                           Force Cable      RSI   14 181.71%       1,387      34   104     89   2024-07-14  2025-06-29
  Large Trimark Armor Pump I Blueprint      RSI   14 179.70% 160,874,864      43    77     76   2024-06-02  2025-07-10
                 Heat Sink I Blueprint      RSI   14 157.42%  50,877,378      37    88     87   2024-06-05  2025-07-06
     Small Gravity Capacitor Upgrade I      RSI    7  94.53%     764,130      34   161     64   2024-07-14  2025-07-12
  Large Trimark Armor Pump I Blueprint      RSI    7  82.40%  96,004,936      41   110     84   2024-06-02  2025-07-10
 

In [28]:
import pandas as pd
import numpy as np
from tqdm import tqdm

# --- Load robust strategies (>5% test ROI) ---
signals_path = r"C:\Users\Jouke\Documents\evedata-logger\output\signals_robust_RSI_gt5pct.csv"
signals_df = pd.read_csv(signals_path, parse_dates=['date'])

# --- Load full market history for fast price lookup ---
history_path = r"C:\Users\Jouke\Documents\evedata-logger\output\update_2025-07-13.csv"
history = pd.read_csv(history_path, parse_dates=['date'])
history['key'] = list(zip(history['type_name'], history['date']))
price_dict = dict(zip(history['key'], history['average']))
latest_price_dict = history.sort_values('date').groupby('type_name')['average'].last().to_dict()

# --- Portfolio Sim Parameters ---
INITIAL_BALANCE = 1_000_000_000  # 1 billion ISK
strategy_count = signals_df['item'].nunique()
signal_count = len(signals_df)
stake_per_signal = INITIAL_BALANCE / signal_count

# --- Simulate Portfolio Growth (Vanilla, Compounded) ---
portfolio = []
balance = INITIAL_BALANCE
equity_curve = []

signals_df = signals_df.sort_values(['date', 'item']).reset_index(drop=True)
open_trades = {}

dates = signals_df['date'].sort_values().unique()

pbar = tqdm(total=len(dates), desc="Simulating portfolio", ncols=80)

for d in dates:
    todays_signals = signals_df[signals_df['date'] == d]
    for idx, sig in todays_signals.iterrows():
        key = (sig['item'], sig['window'])
        price = sig['price']
        if sig['action'] == 'BUY':
            open_trades[key] = {
                'buy_price': price,
                'open_date': sig['date'],
                'stake': stake_per_signal
            }
        elif sig['action'] == 'SELL' and key in open_trades:
            trade = open_trades.pop(key)
            profit = (price - trade['buy_price']) * (trade['stake'] / trade['buy_price'])
            balance += profit  # Compound profit
            portfolio.append({
                'item': sig['item'],
                'window': sig['window'],
                'open': trade['open_date'],
                'close': sig['date'],
                'buy': trade['buy_price'],
                'sell': price,
                'stake': trade['stake'],
                'profit': profit
            })
    # Portfolio value: all open trades marked-to-market + cash
    mtm = 0
    for key, trade in open_trades.items():
        last_price = price_dict.get((key[0], d), latest_price_dict.get(key[0], 0))
        if trade['buy_price'] > 0:
            mtm += (last_price - trade['buy_price']) * (trade['stake'] / trade['buy_price'])
    equity_curve.append({'date': d, 'equity': balance + mtm})
    pbar.update(1)

pbar.close()

# --- Save and Show Results ---
equity_df = pd.DataFrame(equity_curve)
portfolio_df = pd.DataFrame(portfolio)
equity_df.to_csv(r"C:\Users\Jouke\Documents\evedata-logger\output\vanilla_portfolio_equity.csv", index=False)
portfolio_df.to_csv(r"C:\Users\Jouke\Documents\evedata-logger\output\vanilla_portfolio_trades.csv", index=False)
print(f"\nPortfolio simulation done. Final balance: {balance:,.0f} ISK")
print(f"Total trades: {len(portfolio_df)}. Full equity curve saved.")


Simulating portfolio: 100%|█████████████████| 406/406 [00:00<00:00, 1386.32it/s]


Portfolio simulation done. Final balance: 11,530,854,748 ISK
Total trades: 298. Full equity curve saved.
